In [1]:
import sqlite3
import numpy
import time
from utils.preprocess_helpers import make_dir_if_not_exists as mkdir, write_to_file as write
import numpy as np
import subprocess
from tqdm import tqdm


_ROOT = "/home/thanhlc/thanhlc/Data/nbl_dataset/"

In [2]:
dataset = _ROOT + 'dataset.db'
with sqlite3.connect(dataset) as conn:
    c = conn.cursor()

In [3]:
destination = 'result/network_inputs/bugloc-%s/' % time.strftime("%d-%m")
mkdir(destination)
print(destination)

result/network_inputs/bugloc-06-04/


In [4]:
eval_set = np.load(_ROOT + 'data/eval_set.npy', allow_pickle=True).item()
eval_dict = {}
for problem_id in eval_set:
    for program_id, row in eval_set[problem_id].items():
        eval_dict[program_id] = row

eval_set_program_ids = set(eval_dict.keys())
print('len(eval_set_program_ids):', len(eval_set_program_ids))

len(eval_set_program_ids): 2167


In [5]:
query='''SELECT p.program_id, program, test_id, t.verdict FROM 
        programs p INNER JOIN orgsource o ON o.program_id = p.program_id
        INNER JOIN problems q ON o.problem_id = q.problem_id 
        INNER JOIN test_runs t ON t.program_id = p.program_id
        INNER JOIN test_run_summary trs ON trs.program_id = p.program_id
        WHERE trs.verdict<>"ALL_FAIL" AND q.problem_id=?;'''

#test query
# problem_id = 3024
# for row in c.execute(query, (problem_id,)):
#     program_id, program, test_id, verict = row
#     print(program)
#     path = "data/{}/{}.c".format(problem_id, program_id)
#     with open (path, "w") as f:
#         f.write(program)
#     break
problem_ids = [str(row[0]) for row in c.execute('SELECT DISTINCT problem_id FROM orgsource;')]
print('len(problem_ids)', len(problem_ids), problem_ids[0])

len(problem_ids) 29 3024


In [15]:
max_programs_per_test_case_result = 700

test_wise_counts = {}
eval_test_wise_counts = {}
all_data = {}
all_eval_data = {}
test_id_to_problem_id_map = {}

with sqlite3.connect(dataset) as conn:
    c = conn.cursor()

for problem_id in problem_ids:
        test_wise_counts[problem_id] = {}
        eval_test_wise_counts[problem_id] = {}
        this_data_len = 0
        this_eval_data = []
        all_data[problem_id] = {}
        for row in c.execute(query, (problem_id,)):
            program_id = row[0]
            program = row[1]
            test_id = str(row[2])
            verdict = row[3]
            
            ## Just run one time
#             path = _ROOT + "sources/{}/".format(problem_id)
#             mkdir(path)
#             write(row[1], path + str(program_id) + ".c")

            if test_id in test_id_to_problem_id_map:
                assert test_id_to_problem_id_map[test_id] == problem_id
            test_id_to_problem_id_map[test_id] = problem_id
            
            if test_id not in test_wise_counts[problem_id]:
                test_wise_counts[problem_id][test_id] = {0:[], 1:[]}
                eval_test_wise_counts[problem_id][test_id] = {0:[], 1:[]}
                all_data[problem_id][test_id] = {0:[], 1:[]}

            if program_id in eval_set_program_ids:
                this_eval_data += [(program_id, test_id, verdict)]
                eval_test_wise_counts[problem_id][test_id][verdict] += [program_id]
            elif len(test_wise_counts[problem_id][test_id][verdict]) < max_programs_per_test_case_result:
                all_data[problem_id][test_id][verdict] += [(program_id, test_id, verdict)]
                test_wise_counts[problem_id][test_id][verdict] += [program_id]
                this_data_len += 1
                
        all_eval_data[problem_id] = this_eval_data
        print('problem_id: %6s' % problem_id, '#examples:', this_data_len, '#eval_examples:', len(this_eval_data))
        
total = 0
for problem_id in all_data:
    for test_id in all_data[problem_id]:
        for verdict in all_data[problem_id][test_id]:
            total += len(all_data[problem_id][test_id][verdict])
print('total', total)
print(all_data["3024"])
c.close()
conn.close()

problem_id:   3024 #examples: 7658 #eval_examples: 1848
problem_id:   3035 #examples: 10138 #eval_examples: 730
problem_id:   3018 #examples: 7983 #eval_examples: 828
problem_id:   3003 #examples: 9602 #eval_examples: 104
problem_id:   3026 #examples: 5136 #eval_examples: 390
problem_id:   3019 #examples: 4590 #eval_examples: 516
problem_id:   3032 #examples: 6569 #eval_examples: 357
problem_id:   3002 #examples: 6409 #eval_examples: 490
problem_id:   3008 #examples: 10155 #eval_examples: 927
problem_id:   3043 #examples: 8475 #eval_examples: 972
problem_id:   3016 #examples: 9536 #eval_examples: 510
problem_id:   3037 #examples: 4752 #eval_examples: 801
problem_id:   3047 #examples: 7460 #eval_examples: 280
problem_id:   3007 #examples: 3930 #eval_examples: 390
problem_id:   3009 #examples: 6589 #eval_examples: 609
problem_id:   3049 #examples: 11763 #eval_examples: 2790
problem_id:   3030 #examples: 7830 #eval_examples: 90
problem_id:   3055 #examples: 12387 #eval_examples: 1170
prob

In [7]:
for problem_id in all_data:
    print('problem_id:', problem_id)
    for test_id in test_wise_counts[problem_id]:
        print(test_id.replace('.txt', '').replace('IN_', ''),)
        print('fail:', len(test_wise_counts[problem_id][test_id][0]),)
        print('pass:', len(test_wise_counts[problem_id][test_id][1]))      

problem_id: 3024
40013
fail: 273
pass: 700
40024
fail: 553
pass: 455
40078
fail: 250
pass: 700
40224
fail: 273
pass: 700
40297
fail: 546
pass: 462
40331
fail: 248
pass: 700
40394
fail: 199
pass: 700
40410
fail: 199
pass: 700
problem_id: 3035
40021
fail: 527
pass: 529
40082
fail: 392
pass: 664
40139
fail: 390
pass: 666
40205
fail: 382
pass: 674
40212
fail: 393
pass: 663
40237
fail: 394
pass: 662
40294
fail: 248
pass: 700
40312
fail: 224
pass: 700
40352
fail: 265
pass: 700
40383
fail: 265
pass: 700
problem_id: 3018
40049
fail: 431
pass: 456
40108
fail: 312
pass: 575
40274
fail: 360
pass: 527
40334
fail: 308
pass: 579
40335
fail: 313
pass: 574
40426
fail: 311
pass: 576
40444
fail: 332
pass: 555
40447
fail: 342
pass: 545
40449
fail: 459
pass: 428
problem_id: 3003
40006
fail: 700
pass: 531
40015
fail: 700
pass: 279
40141
fail: 231
pass: 700
40156
fail: 391
pass: 700
40362
fail: 686
pass: 700
40409
fail: 700
pass: 693
40453
fail: 700
pass: 491
40460
fail: 700
pass: 700
problem_id: 3026
40002

In [8]:
for problem_id in all_eval_data:
    print('problem_id:', problem_id)
    for test_id in eval_test_wise_counts[problem_id]:
        print(test_id.replace('.txt', '').replace('IN_', ''),) 
        print('fail:', len(eval_test_wise_counts[problem_id][test_id][0]),)
        print('pass:', len(eval_test_wise_counts[problem_id][test_id][1]))

problem_id: 3024
40013
fail: 36
pass: 195
40024
fail: 202
pass: 29
40078
fail: 30
pass: 201
40224
fail: 36
pass: 195
40297
fail: 197
pass: 34
40331
fail: 30
pass: 201
40394
fail: 27
pass: 204
40410
fail: 27
pass: 204
problem_id: 3035
40021
fail: 28
pass: 45
40082
fail: 26
pass: 47
40139
fail: 28
pass: 45
40205
fail: 27
pass: 46
40212
fail: 33
pass: 40
40237
fail: 33
pass: 40
40294
fail: 13
pass: 60
40312
fail: 11
pass: 62
40352
fail: 9
pass: 64
40383
fail: 9
pass: 64
problem_id: 3018
40049
fail: 63
pass: 29
40108
fail: 9
pass: 83
40274
fail: 28
pass: 64
40334
fail: 24
pass: 68
40335
fail: 8
pass: 84
40426
fail: 24
pass: 68
40444
fail: 10
pass: 82
40447
fail: 21
pass: 71
40449
fail: 63
pass: 29
problem_id: 3003
40006
fail: 3
pass: 10
40015
fail: 7
pass: 6
40141
fail: 1
pass: 12
40156
fail: 3
pass: 10
40362
fail: 1
pass: 12
40409
fail: 2
pass: 11
40453
fail: 7
pass: 6
40460
fail: 1
pass: 12
problem_id: 3026
40002
fail: 12
pass: 53
40134
fail: 27
pass: 38
40218
fail: 16
pass: 49
40344
fai

In [9]:
#gen test coverage
#bash coverage run.sh